# Calculating Average Travel Time to Other Destination Categories
- Create a feature class for each destination type (healthcare, higher education, essential services) that contains the point locations, types, and capacity data.
- For each TAZ, find out how many (and ideally which) destinations of each type are within a buffer of the TAZ centroid.
- Flag each TAZ with whether it meets the threshold for the number of destinations within a buffer around the TAZ centroid for the type of destination.
- Use model skims for travel times from TAZ centroid to TAZ centroid for all of the following modes: Drive (SOV as proxy), Transit, Rapid Transit, Bus, Walk.
- Then pick the minimum time to reach a flagged TAZ and weight by TAZ population and use that for calculation of the average travel time for MPO.
- Then flag origin TAZs as whether their minimum distance to a flagged TAZ is within the average threshold for MPO.


Environment: base_py_37_omx_geop


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import openmatrix as omx

In [33]:
##Import all the CSVs that contain skims 
Walk_skim= omx.open_file('M:/LRTP/LRTP_AvgTT/out/Walk_skim.omx','r')
#Import AM
DAT_Boat_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_RapidTransit_tr_skim2.omx','r')
WAT_Transit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/WAT_for_All_tr_skim.omx','r')
SOV_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/SOV_skim.omx','r')

#put AM CSVs into Dictionary
skims_AM = {'DAT_Boat':DAT_Boat_skim_AM, 'DAT_CR':DAT_CommRail_skim_AM, 
            'DAT_LB':DAT_LocalBus_skim_AM, 'DAT_RT':DAT_RapidTransit_skim_AM, 
            'WAT_TR':WAT_Transit_skim_AM, 'SOV':SOV_skim_AM, 'Walk': Walk_skim}

#Import MD
DAT_Boat_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Rapid_Transit_tr_skim.omx','r')
WAT_Transit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\WAT_for_All_tr_skim.omx','r')
SOV_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\SOV_skim.omx','r')

#put MD CSVs into Dictionary
skims_MD = {'DAT_Boat':DAT_Boat_skim_MD, 'DAT_CR':DAT_CommRail_skim_MD, 
            'DAT_LB':DAT_LocalBus_skim_MD, 'DAT_RT':DAT_RapidTransit_skim_MD, 
            'WAT_TR':WAT_Transit_skim_MD, 'SOV':SOV_skim_MD, 'Walk': Walk_skim}

In [34]:
##Import TAZ tables for Essential Services, Healthcare, and Higher Ed. 
dest_TAZs = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/MPO_TAZ_ES_1m2.csv',header=0, sep=',',
                        usecols=['taz', 'ES_FLAG_ALL','HIED_Count5', 'HLTH_COUNT1m'])
#create table with column for each destination type that flags (0,1) which TAZs pass the destination threshold
dest_TAZs['HIED_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HIED_Count5']]
dest_TAZs['HLTH_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HLTH_COUNT1m']]

#bring in population data for later
tot_pop = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/TAZ_Pop_CTPS.csv', header=0, sep=',', usecols=['TAZ_ID', 'Tot_Pop'])
dest_TAZs = dest_TAZs.merge(tot_pop, how='left', left_on='taz', right_on ='TAZ_ID')

#dest_TAZs

In [35]:
#make list of total TAZs (this is for row and column names for matrices)
tazs = dest_TAZs['taz'].tolist()
tazs.sort()
#make lists of only destination TAZs (this is for averaging)
ES_list = dest_TAZs[dest_TAZs['ES_FLAG_ALL']==1]['taz'].tolist()
HLTH_list = dest_TAZs[dest_TAZs['HLTH_FLAG']==1]['taz'].tolist()
HIED_list = dest_TAZs[dest_TAZs['HIED_FLAG']==1]['taz'].tolist()
len(HIED_list)

1594

In [36]:
#import trips matrices
AM_trips = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/AfterSC_Final_AM_Tables.omx','r')
MD_trips = omx.open_file('M:/LRTP/LRTP_AvgTT/out/MD/AfterSC_Final_MD_Tables.omx','r')
#turn into dataframes
AM_trips_DAT = pd.DataFrame((np.array(AM_trips['DAT_Boat'])+np.array(AM_trips['DAT_CR'])+
                             np.array(AM_trips['DAT_LB']) +np.array(AM_trips['DAT_RT']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
AM_trips_WAT = pd.DataFrame(np.array(AM_trips['WAT']))
                            
MD_trips_DAT = pd.DataFrame((np.array(MD_trips['DAT_Boat'])+np.array(MD_trips['DAT_CR'])+
                             np.array(MD_trips['DAT_LB']) +np.array(MD_trips['DAT_RT']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
MD_trips_WAT = pd.DataFrame(np.array(MD_trips['WAT']))   

#close omx
AM_trips.close()
MD_trips.close()

#make trip total tables for transit so can do weighted average
AM_trips_AT = AM_trips_DAT + AM_trips_WAT
MD_trips_AT = MD_trips_DAT + MD_trips_WAT

In [37]:
##Sum the Skims to get Travel Time per Mode by Time of Day (loop through both AM and MD dictionaries)
AM_Tables = {}
MD_Tables = {}

for x in skims_AM.keys():
    if x != 'SOV' and x != 'Walk' and x != 'WAT_TR':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['Access Drive Time'])+np.array(skims_AM[x]['Access Walk Time'])
                        +np.array(skims_AM[x]['Dwelling Time'])+np.array(skims_AM[x]['Egress Drive Time'])
                        +np.array(skims_AM[x]['Egress Walk Time'])+np.array(skims_AM[x]['In-Vehicle Time'])
                        +np.array(skims_AM[x]['Initial Wait Time'])+np.array(skims_AM[x]['Transfer Penalty Time'])
                        +np.array(skims_AM[x]['Transfer Wait Time'])+np.array(skims_AM[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'WAT_TR':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['Access Walk Time'])
                        +np.array(skims_AM[x]['Dwelling Time'])
                        +np.array(skims_AM[x]['Egress Walk Time'])+np.array(skims_AM[x]['In-Vehicle Time'])
                        +np.array(skims_AM[x]['Initial Wait Time'])+np.array(skims_AM[x]['Transfer Penalty Time'])
                        +np.array(skims_AM[x]['Transfer Wait Time'])+np.array(skims_AM[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'SOV':
        AM_Tables[x] = pd.DataFrame(np.array(skims_AM[x]['CongTime_wTerminalTimes'])[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'Walk':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['WalkTime']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        AM_Tables[x][AM_Tables[x] > 60] = None
        
for x in skims_MD.keys():
    if x != 'SOV' and x != 'Walk' and x != 'WAT_TR':
        MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['Access Drive Time'])+np.array(skims_MD[x]['Access Walk Time'])
                        +np.array(skims_MD[x]['Dwelling Time'])+np.array(skims_MD[x]['Egress Drive Time'])
                        +np.array(skims_MD[x]['Egress Walk Time'])+np.array(skims_MD[x]['In-Vehicle Time'])
                        +np.array(skims_MD[x]['Initial Wait Time'])+np.array(skims_MD[x]['Transfer Penalty Time'])
                        +np.array(skims_MD[x]['Transfer Wait Time'])+np.array(skims_MD[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'WAT_TR':
        MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['Access Walk Time'])
                        +np.array(skims_MD[x]['Dwelling Time'])
                        +np.array(skims_MD[x]['Egress Walk Time'])+np.array(skims_MD[x]['In-Vehicle Time'])
                        +np.array(skims_MD[x]['Initial Wait Time'])+np.array(skims_MD[x]['Transfer Penalty Time'])
                        +np.array(skims_MD[x]['Transfer Wait Time'])+np.array(skims_MD[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'SOV':
        MD_Tables[x] = pd.DataFrame(np.array(skims_MD[x]['CongTime_wTerminalTimes'])[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'Walk':
        MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['WalkTime']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        MD_Tables[x][MD_Tables[x] > 60] = None
    


C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in add
  # This is added back by InteractiveShellApp.init_path()
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in add
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in add
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in add


In [38]:
#close everything! 
Walk_skim.close()

DAT_Boat_skim_AM.close()
DAT_CommRail_skim_AM.close()
DAT_LocalBus_skim_AM.close()
DAT_RapidTransit_skim_AM.close()
WAT_Transit_skim_AM.close()
SOV_skim_AM.close()

DAT_Boat_skim_MD.close()
DAT_CommRail_skim_MD.close()
DAT_LocalBus_skim_MD.close()
DAT_RapidTransit_skim_MD.close()
WAT_Transit_skim_MD.close()
SOV_skim_MD.close()

In [40]:
AM_Tables['SOV']

,1,2,3,4,5,6,7,8,9,10,...,2636,2637,2638,2639,2640,2641,2642,2643,2644,2645
1,12.956330,13.643062,13.173339,14.308445,14.404925,14.994924,15.425624,15.913063,15.545771,16.791195,...,55.134129,56.081024,59.575680,61.106041,61.432808,58.969299,56.629562,56.550686,56.036140,53.349316
2,14.328058,13.031709,12.954587,14.101412,13.360558,14.787889,14.381256,14.868696,14.501403,15.746826,...,54.089760,55.036659,58.531311,60.061672,60.388439,57.924931,55.585194,55.506317,54.991772,52.304951
3,14.579362,14.713282,11.080709,12.191051,11.636251,12.877529,12.656950,13.144391,12.777098,14.210814,...,53.082787,54.029686,57.524338,59.054699,59.381466,56.917957,54.578220,54.499344,53.984798,51.297977
4,16.783459,16.917377,12.629704,11.151484,11.763741,12.278058,12.515465,13.002905,12.635612,14.069328,...,54.838863,55.785759,59.280415,60.810776,61.137543,58.674034,56.334297,56.255421,55.740875,53.054050
5,15.900571,16.034492,11.746818,12.616402,10.915481,13.129595,11.812959,12.300400,11.933106,13.366822,...,53.955975,54.902870,58.397526,59.927887,60.254654,57.791145,55.451408,55.372532,54.857986,52.171162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,100.095909,99.735359,96.432137,97.009842,95.512566,94.778748,96.070679,95.046120,95.032318,93.998604,...,12.640846,9.015627,7.432114,9.653997,6.740612,4.141993,5.635468,8.525560,10.164310,17.124113
2642,97.106026,96.745476,93.442253,94.019958,92.522682,91.788864,93.080795,92.056236,92.042435,91.008720,...,11.385929,7.113874,6.484854,10.083163,7.850987,5.150562,3.643444,5.535677,7.174427,14.134228
2643,96.241356,95.880806,92.577583,93.155289,91.658012,90.924194,92.216125,91.191566,91.177765,90.144051,...,10.957171,6.613282,9.173223,12.771533,10.852390,8.388882,5.606019,5.421509,6.309753,13.269554
2644,94.726379,94.365829,91.062607,91.640312,90.143036,89.409218,90.701149,89.676590,89.662788,88.629074,...,8.761540,6.382905,9.973005,13.680373,11.817974,9.354466,7.014727,6.935852,5.506815,11.572702


In [41]:
#make DAT tables (get min for each cell!)
AM_Tables['AM_DAT'] = np.fmin(np.fmin(AM_Tables['DAT_Boat'],AM_Tables['DAT_CR']), np.fmin(AM_Tables['DAT_LB'],AM_Tables['DAT_RT']))
MD_Tables['MD_DAT'] = np.fmin(np.fmin(MD_Tables['DAT_Boat'],MD_Tables['DAT_CR']), np.fmin(MD_Tables['DAT_LB'],MD_Tables['DAT_RT']))

#Create new table in dictionary: All Transit
    #take the average - weight it by # of trips for each type of transit
#AM_Tables['AM_Transit'] = ((AM_Tables['AM_DAT']*AM_trips_DAT)+(AM_Tables['AM_WAT']*AM_trips_WAT))/(AM_trips_AT)
#MD_Tables['MD_Transit'] = ((MD_Tables['MD_DAT']*MD_trips_DAT)+(MD_Tables['MD_WAT']*MD_trips_WAT))/(MD_trips_AT)
    #take the lowest
#AM_Tables['AM_TR'] = np.fmin(AM_Tables['AM_DAT'], AM_Tables['WAT_TR'])
#MD_Tables['MD_TR'] = np.fmin(MD_Tables['MD_DAT'], MD_Tables['WAT_TR'])
#use these if no drive access transit
AM_Tables['AM_TR'] = AM_Tables['WAT_TR']
MD_Tables['MD_TR'] = MD_Tables['WAT_TR']

In [42]:
##Calculate Averages
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.

    #want to end up with a table for each mode for each dest type with these columns: Origin TAZ, Dest TAZ, Time, Tot_Pop
    #need to do AM for Healthcare and Essential Services, do MD for HiEd. (three tables per mode)
Avgs = {}
#Transit - ES
#SOV - ES
#Walking - ES
for x in ['AM_TR', 'SOV', 'Walk']:
    Avgs['ES_'+x] = AM_Tables[x].loc[:,ES_list] #restrict columns to just what is an ES destination TAZ
    Avgs['ES_'+x]['Min_Dest_TAZ'] = Avgs['ES_'+x].idxmin(axis=1) #new column with the column name of the smallest time in each row
    Avgs['ES_'+x]['TravelTime'] = Avgs['ES_'+x].min(1) #new column with the min val in each row (corresponds with dest TAZ above)
    Avgs['ES_'+x] = Avgs['ES_'+x].reset_index() #turn index into origin taz field
    Avgs['ES_'+x] = Avgs['ES_'+x].rename(columns = {'index':'origin taz'})

    Avgs['ES_'+x] = Avgs['ES_'+x][['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table
#Transit - Healthcare
#Walking - Healthcare
#SOV - Healthcare
for x in ['AM_TR', 'SOV', 'Walk']:
    Avgs['HLTH_'+x] = AM_Tables[x].loc[:,HLTH_list] #restrict columns to just what is an HLTH destination TAZ
    Avgs['HLTH_'+x]['Min_Dest_TAZ'] = Avgs['HLTH_'+x].idxmin(axis=1) #new column with the column name of the smallest time in each row
    Avgs['HLTH_'+x]['TravelTime'] = Avgs['HLTH_'+x].min(1) #new column with the min val in each row (corresponds with dest TAZ above)
    Avgs['HLTH_'+x] = Avgs['HLTH_'+x].reset_index() #turn index into origin taz field
    Avgs['HLTH_'+x] = Avgs['HLTH_'+x].rename(columns = {'index':'origin taz'})

    Avgs['HLTH_'+x] = Avgs['HLTH_'+x][['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table

#SOV - HiEd(MD)
#Transit - HiEd (MD)
#Walking - HiEd
for x in ['MD_TR', 'SOV', 'Walk']:
    Avgs['HIED_'+x] = MD_Tables[x].loc[:,HIED_list] #restrict columns to just what is an HiED destination TAZ
    Avgs['HIED_'+x]['Min_Dest_TAZ'] = Avgs['HIED_'+x].idxmin(axis=1) #new column with the column name of the smallest time in each row
    Avgs['HIED_'+x]['TravelTime'] = Avgs['HIED_'+x].min(1) #new column with the min val in each row (corresponds with dest TAZ above)
    Avgs['HIED_'+x] = Avgs['HIED_'+x].reset_index() #turn index into origin taz field
    Avgs['HIED_'+x] = Avgs['HIED_'+x].rename(columns = {'index':'origin taz'})

    Avgs['HIED_'+x] = Avgs['HIED_'+x][['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table


#if want to not restrict to just transit (e.g. if O and D are same TAZ)
#ES_RT.loc[ES_RT['origin taz'].isin(ES_list), 'Min_Dest_TAZ'] = ES_RT['origin taz'] #if taz is already a dest taz, set D taz to O taz
#ES_RT.loc[ES_RT['origin taz'] == ES_RT['Min_Dest_TAZ'], 'TravelTime'] = None #if the taz is a dest taz, set TT to null
#note here: so there will never be a transit value if the dest taz is the same as the origin taz
#will need to replace with Walk or SOV here - depends on DAT or WAT?
Avgs['HIED_MD_TR']

,origin taz,Min_Dest_TAZ,TravelTime
0,1,202.0,12.735808
1,2,3.0,3.000000
2,3,4.0,4.000000
3,4,3.0,3.000000
4,5,3.0,3.000000
...,...,...,...
1896,2641,NaN,NaN
1897,2642,NaN,NaN
1898,2643,NaN,NaN
1899,2644,NaN,NaN


In [43]:
##Calculate Averages PART 2
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.
#actually calculate averages
AvgNums = {}
for x in Avgs.keys():
    Avgs[x] = Avgs[x].merge(dest_TAZs, how = 'left', left_on = 'origin taz', right_on = 'taz') #merge to have all data in one table
    Avgs[x]['TT_Pop'] = Avgs[x]['TravelTime']*Avgs[x]['Tot_Pop'] #calculate time * pop (weight)
    avg = np.nansum(Avgs[x]['TT_Pop'])/np.nansum(Avgs[x]['Tot_Pop']) #make the actual avg, excluding nan values
    AvgNums[x+'_Avg'] = avg #add avg to dictionary with id
    Avgs[x]['Avg'] = avg #add a avg field for calc later
AvgNums

{'ES_AM_TR_Avg': 23.055239813307843,
 'ES_SOV_Avg': 5.628888017575303,
 'ES_Walk_Avg': 8.66032791433802,
 'HLTH_AM_TR_Avg': 26.14402159508965,
 'HLTH_SOV_Avg': 6.4588366804420065,
 'HLTH_Walk_Avg': 10.006462587020875,
 'HIED_MD_TR_Avg': 25.4199342019939,
 'HIED_SOV_Avg': 6.466366023051868,
 'HIED_Walk_Avg': 7.946976722620972}

In [30]:
##Make Flag Tables
#if the TAZ has access to a destination TAZ within the MPO average for that mode, flag.
    #How do I make this into a table?  - maybe copy the TAZ to TAZ table and clear its contents, then just go down the column of all the destination TAZs and query the skim table for that mode and TOD.
    #where column name is in list (aka field in the table that flags destination TAZs for each type)
for x in Avgs.keys():
    Avgs[x]['AvgTT_Flag'] = np.where(Avgs[x]['TravelTime'] <= Avgs[x]['Avg'], 1, 0)


In [28]:
Avgs['HLTH_Walk'].max()

origin taz        2645.000000
Min_Dest_TAZ      2642.000000
TravelTime          51.282684
taz               2645.000000
ES_FLAG_ALL          1.000000
HIED_Count5         49.000000
HLTH_COUNT1m        23.000000
HIED_FLAG            1.000000
HLTH_FLAG            1.000000
TAZ_ID            2645.000000
Tot_Pop           5151.000000
TT_Pop          184343.428539
Avg                 10.006463
dtype: float64

In [32]:
for x in Avgs.keys():
    Avgs[x].to_csv('M:\LRTP\LRTP_AvgTT\Avgs_Walk\Avg_'+x+'.csv',sep = ',')

In [ ]:
#DO NOT RUN (works - use for future)

##Calculate Averages
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.

    #want to end up with a table for each mode for each dest type with these columns: Origin TAZ, Dest TAZ, Time, Tot_Pop
    #need to do AM for Healthcare and Essential Services, do MD for HiEd. (three tables per mode)
ES_RT = AM_Tables['DAT_RT'].loc[:,ES_list] #restrict columns to just what is an ES destination TAZ
ES_RT['Min_Dest_TAZ'] = ES_RT.idxmin(axis=1) #new column with the column name of the smallest time in each row
ES_RT['TravelTime'] = ES_RT.min(1) #new column with the min val in each row (corresponds with dest TAZ above)
ES_RT = ES_RT.reset_index() #turn index into origin taz field
ES_RT = ES_RT.rename(columns = {'index':'origin taz'})

ES_RT = ES_RT[['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table

#if want to not restrict to just transit (e.g. if O and D are same TAZ)
#ES_RT.loc[ES_RT['origin taz'].isin(ES_list), 'Min_Dest_TAZ'] = ES_RT['origin taz'] #if taz is already a dest taz, set D taz to O taz
#ES_RT.loc[ES_RT['origin taz'] == ES_RT['Min_Dest_TAZ'], 'TravelTime'] = None #if the taz is a dest taz, set TT to null
#note here: so there will never be a transit value if the dest taz is the same as the origin taz
#will need to replace with Walk or SOV here - depends on DAT or WAT?
ES_RT